In [ ]:
import os
import sys
import time

import GCode
import GRBL
import matplotlib.pyplot as plt
import numpy as np

from utils import picture

In [12]:
sys.path.append(os.path.abspath(".."))

# Draw a Brick Pattern

Attempt at programmatically making a brick pattern.

All units in mm. ```1``` = ```1 mm```.

> Jenga is played with 54 wooden blocks. Each block is three times longer than its width, and one fifth as thick as its length 1.5 cm × 2.5 cm × 7.5 cm (0.59 in × 0.98 in × 2.95 in).

https://en.wikipedia.org/wiki/Jenga

In [13]:
# Standard brick dimensions.
BrickHeight = 65  # [mm]
BrickLength = 225  # [mm]
BrickDepth = 12.5  # [mm]
BrickRatio = 215 / 65  # [dimensionless]


In [14]:
# Jenga
BlockHeight = 25  # [mm]
BlockLength = 75  # [mm]


In [15]:
# Drawing configuration.
# How many rows of bricks to draw on the block.
N_BrickRows = 3  # [dimensionless]

# Dimensions of a 'brick' projected onto the block of wood.
H_Block_Brick = BlockHeight / N_BrickRows  # [mm]
L_Block_Brick = H_Block_Brick * BrickRatio  # [mm]


# Code:

In [16]:
flip = np.array([[1, 1], [1, 0]])
transform_tuple = (
    np.eye(2),  # Identity matrix, do nothing.
    np.eye(2),  # Do nothing, for debugging.
    flip,  # Flip the matrix, reduces travel time.
)
vertical_brick_lines_tuple = (
    np.arange(L_Block_Brick, BlockLength, L_Block_Brick),  # Odd rows.
    np.arange(L_Block_Brick / 2, BlockLength, L_Block_Brick),  # Even rows.
)
horizontal_brick_lines = np.linspace(
    0, BlockHeight, N_BrickRows, endpoint=False
)


Lines parallel to the X-axis.
Separates rows of bricks.

In [17]:
horizontal_brick_lines


array([ 0.        ,  8.33333333, 16.66666667])

Lines parallel to the Y-axis.

In [18]:
vertical_brick_lines_tuple


(array([27.56410256, 55.12820513]),
 array([13.78205128, 41.34615385, 68.91025641]))

In [21]:
BlockBrick = GCode.Program()
BlockBrick.lines = list()

for idx in range(1, len(horizontal_brick_lines)):
    # Top horizontal line that defines each 'brick'
    horizontal_brick_line = horizontal_brick_lines[idx]
    row_line_points = np.array(
        [[0, horizontal_brick_line], [BlockLength, horizontal_brick_line]]
    )
    # Transform to perform on the row points.
    transform = transform_tuple[np.mod(idx, 2)]

    row_line_points = np.matmul(transform, row_line_points)
    line_ = GCode.Line(points=row_line_points)

    BlockBrick.lines.append(line_)


In [22]:
for idx in range(1, len(horizontal_brick_lines)):

    # Vertical brick line.
    vertical_brick_lines = vertical_brick_lines_tuple[
        np.mod(idx, len(vertical_brick_lines_tuple))
    ]

    start_point_y = horizontal_brick_lines[idx - 1]
    end_point_y = horizontal_brick_lines[idx]

    for idx2, vertical_brick_line in enumerate(vertical_brick_lines):
        transform = transform_tuple[np.mod(idx2, 2)]
        column_line_points = (
            np.array(
                [
                    [vertical_brick_line, start_point_y],
                    [vertical_brick_line, end_point_y],
                ]
            ),
        )
        column_line_points = np.matmul(transform, column_line_points)
        line_ = GCode.Line(column_line_points)
        BlockBrick.lines.append(line_)
# Not drawing the last row of vertical lines.
idx += 1
vertical_brick_lines = vertical_brick_lines_tuple[
    np.mod(idx, len(vertical_brick_lines_tuple))
]

start_point_y = horizontal_brick_lines[idx - 1]
end_point_y = BlockHeight

for idx2, vertical_brick_line in enumerate(vertical_brick_lines):
    transform = transform_tuple[np.mod(idx2, 2)]
    column_line_points = (
        np.array(
            [
                [vertical_brick_line, start_point_y],
                [vertical_brick_line, end_point_y],
            ]
        ),
    )
    column_line_points = np.matmul(transform, column_line_points)
    line_ = GCode.Line(column_line_points)
    BlockBrick.lines.append(line_)


In [24]:
BlockBrick.lines


[Line<cut=75.0mm, F=300, S=150>,
 Line<cut=75.0mm, F=300, S=150>,
 Line<cut=8.33333mm, F=300, S=150>,
 Line<cut=8.33333mm, F=300, S=150>,
 Line<cut=8.33333mm, F=300, S=150>,
 Line<cut=8.33333mm, F=300, S=150>,
 Line<cut=8.33333mm, F=300, S=150>,
 Line<cut=8.33333mm, F=300, S=150>,
 Line<cut=8.33333mm, F=300, S=150>,
 Line<cut=8.33333mm, F=300, S=150>]

In [25]:
BlockBrick.time


262.7764987313262

In [26]:
BlockBrick.dist


655.5529174626523

In [27]:
BlockBrick.generate_gcode()


In [28]:
BlockBrick


Program<lines=10>

In [29]:
cnc = GRBL.GRBL(port="/dev/cnc_3018")
cnc.reset()


(0, ['ok', '', "Grbl 1.1f ['$' for help]", 'ok'])

In [30]:
cnc.status


'<Idle|MPos:-255.900,-136.000,-1.000|Bf:15,127|FS:0,0|WCO:0.000,0.000,0.000>'

In [31]:
BlockBrick.machine = cnc


In [32]:
BlockBrick.run()
